In [41]:
import pandas as pd
import numpy as np
# data from broze 
hourly_df = pd.read_csv("/Users/cgp/Portfolio/WeatherDemand/wede/data/bronze/hourly_weather_data.csv", parse_dates=['datetime'])
daily_df = pd.read_csv("/Users/cgp/Portfolio/WeatherDemand/wede/data/bronze/daily_weather_data.csv", parse_dates=['date', 'sunrise', 'sunset'])

print(f"Hourly: {hourly_df.shape}")
print(f"Daily: {daily_df.shape}")

Hourly: (26520, 9)
Daily: (1105, 3)


In [42]:
hourly_df['date'] = hourly_df['datetime'].dt.date
daily_df['date'] = daily_df['date'].dt.date
silver_df = hourly_df.merge(daily_df, on='date', how='left') #Merging hourly with daily on date

In [43]:
# Dealing with missing values in sunrise and sunset cols
silver_df[silver_df['sunrise'].isna() | silver_df['sunset'].isna()]

,datetime,temperature,humidity,precipitation,rain,snowfall,wind_speed,cloud_cover,is_day,date,sunrise,sunset
26512,2026-01-10 00:00:00+00:00,6.25,88.252045,0.0,0.0,0.0,16.478603,100.0,1.0,2026-01-10,NaT,NaT
26513,2026-01-10 01:00:00+00:00,6.25,86.115130,0.0,0.0,0.0,14.182355,82.0,0.0,2026-01-10,NaT,NaT
26514,2026-01-10 02:00:00+00:00,5.95,84.582690,0.0,0.0,0.0,12.503199,98.0,0.0,2026-01-10,NaT,NaT
26515,2026-01-10 03:00:00+00:00,5.80,83.085970,0.0,0.0,0.0,11.854062,99.0,0.0,2026-01-10,NaT,NaT
26516,2026-01-10 04:00:00+00:00,5.80,80.480410,0.0,0.0,0.0,10.948973,100.0,0.0,2026-01-10,NaT,NaT
26517,2026-01-10 05:00:00+00:00,5.65,79.889550,0.0,0.0,0.0,9.109138,100.0,0.0,2026-01-10,NaT,NaT
26518,2026-01-10 06:00:00+00:00,5.20,80.970220,0.0,0.0,0.0,7.347000,99.0,0.0,2026-01-10,NaT,NaT
26519,2026-01-10 07:00:00+00:00,4.80,81.494710,0.0,0.0,0.0,6.048107,99.0,0.0,2026-01-10,NaT,NaT


In [44]:
silver_df.loc[silver_df['datetime'].dt.date == pd.to_datetime('2026-01-10').date(), 'sunrise'] = pd.to_datetime('2026-01-10 08:06:00-08:00')
silver_df.loc[silver_df['datetime'].dt.date == pd.to_datetime('2026-01-10').date(), 'sunset'] = pd.to_datetime('2026-01-10 16:32:00-08:00')

### Feature Eng

In [45]:
# Time features
silver_df['hour'] = silver_df['datetime'].dt.hour
silver_df['day_of_week'] = silver_df['datetime'].dt.dayofweek  # 0=Mon, 6=Sun
silver_df['month'] = silver_df['datetime'].dt.month
silver_df['is_weekend'] = silver_df['day_of_week'].isin([5, 6]).astype(int)

In [46]:
# Daylight duration (hours)
silver_df['daylight_duration'] = (silver_df['sunset'] - silver_df['sunrise']).dt.total_seconds() / 3600


In [47]:
#0=none, 1=light, 2=moderate, 3=heavy (impacts behavior)
silver_df['precipitation_flag'] = pd.cut(
    silver_df['precipitation'], 
    bins=[-np.inf, 0.5, 2.0, 5.0, np.inf], 
    labels=[0, 1, 2, 3]
).astype(int)

In [48]:
# 0=Cold (<5°C), 1=Cool (5-12°C), 2=Mild (12-18°C), 3=Warm (18-25°C), 4=Hot (>25°C rare)
silver_df['temp_category'] = pd.cut(
    silver_df['temperature'],
    bins=[-np.inf, 5, 12, 18, 25, np.inf],
    labels=[0, 1, 2, 3, 4]
).astype(int)

In [49]:
silver_df['bad_weather_combo'] = ((silver_df['rain'] > 2) & (silver_df['wind_speed'] > 20)).astype(int)
# Boolean feature indicating bad weather conditions --> rain > 2mm and wind speed > 20 km/h

In [50]:
# Peak hours (lunch: 11-13, dinner: 17-20)
silver_df['is_peak_hour'] = silver_df['hour'].isin([11, 12, 13, 17, 18, 19, 20]).astype(int)

# Season (0=Winter, 1=Spring, 2=Summer, 3=Fall)
silver_df['season'] = silver_df['month'].apply(lambda m: 0 if m in [12, 1, 2] else 1 if m in [3, 4, 5] else 2 if m in [6, 7, 8] else 3)

In [51]:
silver_df = silver_df.drop(columns=['date'])

print(f"\nProcessed dataset:{silver_df.info()}")
print(f"Columns: {silver_df.columns.tolist()}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26520 entries, 0 to 26519
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   datetime            26520 non-null  datetime64[ns, UTC]
 1   temperature         26520 non-null  float64            
 2   humidity            26520 non-null  float64            
 3   precipitation       26520 non-null  float64            
 4   rain                26520 non-null  float64            
 5   snowfall            26520 non-null  float64            
 6   wind_speed          26520 non-null  float64            
 7   cloud_cover         26520 non-null  float64            
 8   is_day              26520 non-null  float64            
 9   sunrise             26520 non-null  datetime64[ns, UTC]
 10  sunset              26520 non-null  datetime64[ns, UTC]
 11  hour                26520 non-null  int32              
 12  day_of_week         26520 non-nu

In [53]:
silver_df.to_csv("/Users/cgp/Portfolio/WeatherDemand/wede/data/silver/weather_features.csv", index=False)